## Pre-Training [`GPT-Neo`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt_neo/modeling_gpt_neo.py) & [`RoBERTa`](https://github.com/huggingface/transformers/blob/main/src/transformers/models/roberta/modeling_roberta.py) on [`TinyStories`](https://huggingface.co/datasets/roneneldan/TinyStories)

This script shows how to pre-train both models. I put them in one notebook because the majority of the code is shared; but you may want to separate the logic per model. 

In [31]:
# %pip install torch wandb transformers[torch] datasets tqdm 
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


python(30047) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [32]:
import os
import transformers 
print(transformers.__version__)
import torch 
print(torch.__version__)

4.40.1
2.3.0


In [33]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # or "0,1" for multiple GPUs
from typing import Optional, Tuple
# import wandb; wandb.login()

#wandb login with api key
import wandb
wandb.login(key="b18c2e103bce4f9c38f895080cbdfb6284617f99")
import torch
import torch.nn.functional as F
import torch.nn as nn
from transformers import (
    RobertaForMaskedLM, RobertaConfig, RobertaTokenizer,
    GPTNeoForCausalLM, GPTNeoConfig, GPT2TokenizerFast
)
debug_mode = True

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


#### Models 
We consider GPT-Neo and BERT as base transformer architectures. This consists of the following blocks linked via residual connections:

- Embeddings: Map one-hot (sparse) token vectors to a dense vector of length `hidden_size`. Add positional encoding. 
- $n$ Blocks: Contain self-attention and FFNs.
- Head: Map hidden state back to a useful output. 

This specifies some of the model-related hyperparameters. I chose them based on what achieved reasonable performance in the [TinyStories paper](https://arxiv.org/abs/2305.07759), while also being feasible to train on our limited compute budgets. 

In [34]:
config_gpt = dict(

    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   # number of tokens in the vocabulary 
    hidden_size             = 516,      # embedding size (vector length) of each token 
    max_position_embeddings = 512,      # maximum sequence length (context window)

    # BLOCKS (ATTN & FFN)
    num_layers          = 2,                    # number of transformer blocks
    attention_types     = [[["global", "local"], 1]], # (GPT-Neo-specific) global and local attention 
    num_heads           = 4,                    # attention heads
    window_size         = 256,                  # (GPT-Neo-specific) for local attention 
    intermediate_size   = 1024,                 # size of 'up-projection' layer in FFN

    pad_token_id = 0,           # need to specify this for tokenizer interop between models
    segment_len = 16,           # (InfiniAttention) 
)

config_rob = dict(
    
    # EMBEDDING PARAMETERS
    vocab_size              = 10_000,   
    hidden_size             = 516,      
    # we add 1 as RoBERTa uses a special position embedding for the padding token (zero vector)
    max_position_embeddings = config_gpt['max_position_embeddings'] + 1,

    # BLOCKS (of course naming is different in roberta :) )
    num_hidden_layers = config_gpt['num_layers'],
    num_attention_heads = config_gpt['num_heads'],
    intermediate_size=1024,                     

    pad_token_id = 0,
)

config_gpt = GPTNeoConfig(**config_gpt)
config_rob = RobertaConfig(**config_rob)

In [35]:
from selfattentionVanilla import GPTNeoSelfAttentionVanilla
from selfattention import InfiniAttention
class InfiniAttentionGPTNeoForCausalLM(GPTNeoForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        # Assuming attention types are specified in config.attention_layers
        # Replace the standard attention with the specified type
        for block in self.transformer.h:
            block.attn.attention = InfiniAttention(config_gpt, block.attn.attention_type)


In [36]:
# TODO: you should set all pytorch & huggingface seeds here as model initialisation depends on it

gpt = InfiniAttentionGPTNeoForCausalLM(config=config_gpt)
rob = RobertaForMaskedLM(config=config_rob)

print(f'''
    This GPT has {gpt.num_parameters():,} parameters,
     and ROB has {rob.num_parameters():,} parameters.
    ''')

# gpt, rob # uncomment to see model architecture


    This GPT has 9,677,056 parameters,
     and ROB has 9,959,496 parameters.
    


python(30048) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [37]:
from transformers import PreTrainedTokenizer, PretrainedConfig

def get_tokenizer_for_config(Tok: PreTrainedTokenizer, config: PretrainedConfig):
    print(f'The current directory is: {os.getcwd()}')
    tokenizer = Tok.from_pretrained(
        '10k-tok',                                         # our custom tokenizer
        model_max_length=config.max_position_embeddings    # sequence length (context window)
    )

    # we're using our special tokenizer with only 10'000 tokens instead of 50'256
    assert tokenizer.vocab_size == config.vocab_size

    print(f'padding token is {tokenizer.pad_token}')
    print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')
    
    return tokenizer 

tok_gpt = get_tokenizer_for_config(GPT2TokenizerFast, config_gpt)
tok_rob = get_tokenizer_for_config(RobertaTokenizer, config_rob)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.


The current directory is: /Users/laurikeskull/Documents/Programming/tiny-transformers/code/lauri
padding token is <pad>
padding token in config: 0, in tokeniser: 0
The current directory is: /Users/laurikeskull/Documents/Programming/tiny-transformers/code/lauri
padding token is <pad>
padding token in config: 0, in tokeniser: 0


In [38]:
from datasets import load_from_disk 

if debug_mode:
    tokenized_dataset = load_from_disk(f'./tokenized_dataset_small')
else:
    tokenized_dataset = load_from_disk(f'./tokenized_dataset')


train_dataset = tokenized_dataset['train']
eval_dataset  = tokenized_dataset['validation']

assert len(tokenized_dataset['train'][0]['input_ids']) == config_gpt.max_position_embeddings
tokenized_dataset['train'][0]['input_ids'][-10:]
# should be pad tokens (0), given that most short stories are <512 tokens

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

#### Training
Before we get started, you may want to specify which GPU to use. See the first cell in this notebook; make sure to run it before anything else. 

Huggingface provides some powerful (and often confusingly long) APIs for model training. The `TrainingArguments` specifies our hyperparameters, which are used by the `Trainer` taking in the remaining objects (like `model`, `tokenizer`, and `train_dataset`). Specifically:

- `learning_rate` and `num_train_epochs` determine how much the model learns. A higher rate is faster, but more unstable. More epochs (entire passes over the dataset) yields incrementally better results, at the cost of more training time. 
- Batch sizes determine how many samples the model sees in *parallel*. Given `gradient_accumulation_steps=1` and a `batch_size=8`, the model will backpropagate the average loss of 8 samples; if `batch_size=1`, it will average the loss of `gradient_accumulation_steps` samples. It is important to make sure the backpropagated loss is averaged over the same number of samples, when comparing models. 

- `data_collator` batches (and optionally, pads) the input for the model. We have already padded in our `tokenized_dataset`, and leaving this argument empty will automatically batch the inputs. So why do we need it? 

    Glad you asked. This has to do with how the loss is computed in causal language modelling. In our case, we try to predict $p(y | x)$, where $x$ is an input sequence of tokens, and $y$ is the next token following that sequence. Our model, unaware of the target token $y$, outputs $\hat y$. 
    
    For `Trainer` to compute the (cross-entropy) loss, we need to provide it with both $y$ and $\hat y$. The `DataCollatorForLanguageModeling` knows this, and provides the next token $y$ as a separate part of the input, to the `Trainer`.

    The loss is the backbone of backpropagation, which we need to actually improve our model. If this is confusing, please re-watch Karpathy's GPT tutorial. 

If you prefer to write the training loop yourself, check out HF' `run_clm_no_trainer.py` scripts. (`run_mlm_no_trainer.py` for RoBERTa-style masked-language modelling, as opposed to causal language modelling). This can be useful to give you better control over which devices are used for training. 

In [39]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
model_name_for_save = ''
def get_hyperparameters(model, dataset):
    ''' common hyperparameters to give to the trainer '''

    # TRAINING HYPERPARAMETERS 
    batch_size = 16                  # TinyStories uses 80, but I am training locally on my poor M1 Air
    num_train_epochs = 1             # TinyStories doesn't mention
    gradient_accumulation_steps = 16 # TinyStories uses 16

    lr = 5e-4                        # TinyStories uses 5e-4, higher values better for small models

    # future you will thank you for descriptive model names
    # TODO: customise this name such that every model you train has a unique identifier!
    config      = model.config 
    model_name  = '-'.join([
        'INFINI_GPT' if isinstance(model, InfiniAttentionGPTNeoForCausalLM) else (
        'GPT' if isinstance(model, GPTNeoForCausalLM) else 'BERT'),
        f'{model.num_parameters()//1e6:.1f}M',
        f'{config.num_layers if isinstance(model, GPTNeoForCausalLM) else config.num_hidden_layers}L', 
        f'{config.num_heads if isinstance(model, GPTNeoForCausalLM) else config.num_attention_heads}H', 
        f'{config.hidden_size}C',
        f'{config.intermediate_size}I'
    ])
    model_name_for_save = model_name

    _train_steps = len(dataset) // (batch_size * gradient_accumulation_steps)
    eval_steps = _train_steps // 10 # evaluate every 10% of training steps

    return dict(
        model_name = model_name,
        batch_size = batch_size, 
        num_train_epochs = num_train_epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        lr = lr,
        eval_steps = eval_steps
    )

params_gpt = get_hyperparameters(gpt, train_dataset)
params_rob = get_hyperparameters(rob, train_dataset)

In [40]:
def get_trainer(
        model, tokenizer, train_dataset, eval_dataset, output_dir,
        model_name, batch_size, num_train_epochs, gradient_accumulation_steps, lr, eval_steps):
    ''' more general training arguments you likely want to keep fixed'''

    training_args = TrainingArguments(

        seed       = 42,
        # use_cpu    = True, # use GPU if available (not necessarily faster on laptops, but Apple's MPS have good support)

        output_dir = os.path.join(output_dir, model_name),

        # NOTE: training params
        learning_rate    = lr,
        num_train_epochs = num_train_epochs,
        # Use a smaller batch size to fit into GPU RAM. 
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size  = batch_size,
        # You should aim to have the same amount of samples per acc step, in all of your experiments!
        # so, if you increase batch_size, decrease gradient_accumulation_steps by the same factor.
        gradient_accumulation_steps = gradient_accumulation_steps,

        # NOTE: Evaluation params
        # wandb is great for tracking experiments, it will even (try to) save your code nowadays
        evaluation_strategy = 'steps',
        eval_steps = eval_steps,
        save_steps = eval_steps,

        logging_first_step=True,
        logging_steps=100,
        report_to  = 'wandb',
    )

    trainer = Trainer(
        model = model, 
        args = training_args, 
        train_dataset = train_dataset, 
        eval_dataset = eval_dataset,
        data_collator = DataCollatorForLanguageModeling(
            tokenizer, mlm=isinstance(model, RobertaForMaskedLM)),
    )

    # print amount of training steps, and how often the model is evaluated
    print(f'''
    Retrieving Trainer for \033[1m{model_name}\033[0m
        training for {num_train_epochs} epochs, {len(train_dataset)} samples
        {batch_size} batch size, {gradient_accumulation_steps} accumulation steps
        gives {len(train_dataset)//(batch_size * gradient_accumulation_steps)} training steps.
        Evaluating every {eval_steps} steps, {len(eval_dataset)} samples 
        ''')

    return trainer

In [41]:
out_dir = './results/models_baseline/' 

trainer_gpt = get_trainer(gpt, tok_gpt, train_dataset, eval_dataset, out_dir, **params_gpt)
trainer_rob = get_trainer(rob, tok_rob, train_dataset, eval_dataset, out_dir, **params_rob)

python(30049) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(30050) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(30052) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



    Retrieving Trainer for INFINI_GPT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 100 samples
        16 batch size, 16 accumulation steps
        gives 0 training steps.
        Evaluating every 0 steps, 10 samples 
        


python(30053) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



    Retrieving Trainer for BERT-9.0M-2L-4H-516C-1024I
        training for 1 epochs, 100 samples
        16 batch size, 16 accumulation steps
        gives 0 training steps.
        Evaluating every 0 steps, 10 samples 
        


Finally, we can train. 

This configuration takes ≤24hr to pre-train on my M1 Macbook Air with 16GB RAM. Python takes ≤4GB VRAM at a `batch_size=16` and ≤11GB at `batch_size=64`, though they take the same amount of time to train - likely because this processor is not designed to move that much data in and out of RAM constantly. And tbh, the GPU be lacking. If you decide to go the local-training-route, consider [chai](https://github.com/lvillani/chai) to keep your (Apple) laptop awake – there's probably a windows/linux equivalent too. 

In [42]:
def do_train(trainer: Trainer, name: str, out_dir: str): 

    wandb.init(project='tiny-transformers', name=name, group='baseline', config=trainer.args)
    trainer.train()
    trainer.save_model(os.path.join(out_dir, name))

In [43]:
# do_train(trainer_rob, params_rob['model_name'], out_dir)

python(30055) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [44]:
wandb.finish()

python(30057) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
wandb: ERROR Control-C detected -- Run data was not synced


In [ ]:
do_train(trainer_gpt, params_gpt['model_name'], out_dir) 

100%|██████████| 1/1 [00:09<00:00,  9.99s/it]


{'loss': 4.0633, 'grad_norm': 1.3105493783950806, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 9.9787, 'train_samples_per_second': 10.021, 'train_steps_per_second': 0.1, 'train_loss': 4.063251495361328, 'epoch': 1.0}


In [ ]:
# right after training
trained_model = trainer_gpt.model
model_name = params_gpt['model_name']

torch.save(trained_model.state_dict(), f'./results/models_baseline/{model_name}/model_state.pt')

# before evaluation
config_1 = GPTNeoConfig.from_pretrained(f'./results/models_baseline/{model_name}')
custom_model_1 = InfiniAttentionGPTNeoForCausalLM(config=config_1)
trained_state_dict_1 = torch.load(f'./results/models_baseline/{model_name}/model_state.pt')
custom_model_1.load_state_dict(trained_state_dict_1)

python(26049) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(26059) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


<All keys matched successfully>

#### Using your Pre-Trained Model 
Try out your own model on some prompts!

In [ ]:
# I made a small function for pretty printing into paragraphs with line wrapping for readability
import textwrap 
w = textwrap.TextWrapper(replace_whitespace=False, break_long_words=False, width=60, initial_indent='   ', subsequent_indent='  ')
def see(text): print('\n\033[3m' + '\n\n'.join(['\n'.join(w.wrap(line))
                 for line in text.splitlines() if line.strip() != '']) + '\033[0m\n')

python(23767) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = 'GPT-INFINI-9.0MINFINI-2L-4H-516C-1024I'
vanilla = 'GPT-9.0M-2L-4H-516C-1024I'
# model_name = 'BERT-9.0M-2L-4H-516C-1024I' # bert won't work for generation unless you fine-tune it for that task
infin_name = 'INFINI_GPT-9.0M-2L-4H-516C-1024I'
tokenizer = AutoTokenizer.from_pretrained('10k-tok')
# tokenizer_old = AutoTokenizer.from_pretrained('10k-gpt-neo')

tokenizer.pad_token_id = tokenizer.eos_token_id
model = InfiniAttentionGPTNeoForCausalLM.from_pretrained(f'results/models_baseline/{infin_name}', use_safetensors=True)

# model = AutoModelForCausalLM.from_pretrained(f'results/models_baseline/{infin_name}')
state_dict = model.state_dict()
print(state_dict['transformer.h.0.attn.attention.betas'])
# custom_model.load_state_dict(state_dict)
print(type(model))

python(24827) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Some weights of the model checkpoint at results/models_baseline/INFINI_GPT-9.0M-2L-4H-516C-1024I were not used when initializing InfiniAttentionGPTNeoForCausalLM: ['transformer.h.0.attn.attention.biass', 'transformer.h.1.attn.attention.biass']
- This IS expected if you are initializing InfiniAttentionGPTNeoForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing InfiniAttentionGPTNeoForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of InfiniAttentionGPTNeoForCausalLM were not initialized from the model checkpoint at results/models_baseline/INFINI_GPT-9.0M-2L-4H-516C-1024I and

tensor([[[[-1.9247]],

         [[-2.1131]],

         [[-2.6322]],

         [[-0.0114]]]])
<class '__main__.InfiniAttentionGPTNeoForCausalLM'>


In [ ]:
from safetensors import safe_open
import torch

# Path to your .safetensors file
safetensors_file_path = 'results/models_baseline/INFINI_GPT-9.0M-2L-4H-516C-1024I/model.safetensors'
trained_model_safetensor_path = 'results/infini_baseline/checkpoint-8280/model.safetensors'


# Open the safetensors file
with safe_open(safetensors_file_path, framework="pt") as f:
    # List all tensor names
    tensor_names = f.keys()
    print("Tensors in the safetensors file:")
    for name in tensor_names:
        print(name)

    # Load a specific tensor
    tensor_name = tensor_names[0]  # Replace with the name of the tensor you want to load
    tensor = f.get_tensor(tensor_name)
    print(f"Tensor {tensor_name}:")
    print(tensor[0])
    print(tensor.size())


Tensors in the safetensors file:
transformer.h.0.attn.attention.betas
transformer.h.0.attn.attention.k_proj.weight
transformer.h.0.attn.attention.out_proj.bias
transformer.h.0.attn.attention.out_proj.weight
transformer.h.0.attn.attention.q_proj.weight
transformer.h.0.attn.attention.v_proj.weight
transformer.h.0.ln_1.bias
transformer.h.0.ln_1.weight
transformer.h.0.ln_2.bias
transformer.h.0.ln_2.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.1.attn.attention.betas
transformer.h.1.attn.attention.k_proj.weight
transformer.h.1.attn.attention.out_proj.bias
transformer.h.1.attn.attention.out_proj.weight
transformer.h.1.attn.attention.q_proj.weight
transformer.h.1.attn.attention.v_proj.weight
transformer.h.1.ln_1.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_2.bias
transformer.h.1.ln_2.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_proj.bias
t

python(26088) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


#### Inference 
Let's generate a short story like the ones the model has been trained on! You'll notice that the prompt is surrounded by `<s>`, the begin-of-sequence (bos) token, and `</s>` end-of-sequence (eos) / separator (sep) token. This is from the BERT-style tokenisation, making it clear to the model where (one of several) input sequences ends. 

In [ ]:
prompt = 'Once upon a time, there was a little car named Beep. He then '
input_ids = tokenizer.encode(prompt, return_tensors='pt')

output = custom_model_1.generate(
    input_ids,                              # input to the model
    max_length=300,                         # maximum generation length
    eos_token_id=tokenizer.eos_token_id,    # early stopping when eos_token is output

    # num_beams=1,                            # number of beams to use in generation
    temperature=1,
)
# print(output)
output_text = tokenizer.decode(output[0])

# textwrap with indentation on every new paragraph
see(output_text)
output

python(24819) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



   <s>Once upon a time, there was a little car named Beep.
  He then
  </s>.........................................................................................................................................................................................................................................................................................



tensor([[   1, 7454, 2402,  257,  640,   11,  612,  373,  257, 1310, 1097, 3706,
         1355,  538,   13,  679,  788,  220,    2,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,   13,
           13,   13,   13,  

Sounds like me after a few beers too many, but at least the grammar is (mostly) correct. The model also learns some basic reasoning-like associations like being 'so high' allows you to see 'the whole world'. 